# Metodo simplex

Como hemos visto, el algoritmo del metodo simplex consta de pasos que son relativamente sencillos

* poner en forma estandar y cacnonica
* pivotear
* operaciones elementales de matrices
* revisar condicion de paro

Aunque se vea relativamente simple, implementar los pasos puede ser complicado, y si no se tiene cuidado, la complejidad del algoritmo podria incrementar, bajando su eficiencia de tiempo (de por si no hay una complejidad fija para el simplex).


## Boceto del simplex

Ahora que ya sabemos los pasos, vamos ahora a pensar en como poder solucionar el simplex

```
input : forma estandar y canonica
A[-1] = z
A[:][-1] = restricciones


sacar pivote:
    sacar columna, con el numero mas pequeño en Z
    division A_i/b_i y el min es mi fila
    el min de la division tiene que ser  > 0


aplicar operaciones elementales:
    hacer uno la fila de mi pivote
    hacer 0 arriba y abajo del pivote:
        multiplicar por el elemento que se desea hacer 0 y restarserlo a la fila correspondiente
    

revisar condicion de paro:
    que ya no hubiera numeros negativos en Z

si no condicion: regresar a sacar pivote

Regresa: coeficientes de las variables basicas
```

## ¿Cómo podemos representar todo esto en una computadora?


Una matriz es un arreglo de arreglos.

```
x1   x2     RHS
a_11 a_12 | a_13
a_21 a_22 | a_23 
-----------
z_1  z_2  | 0 <- z 
```
En lugar de tomar toda la matriz, tomar  submatriz `A[:-1]` y RHS `A[:-1][-1]` y operaciones elementales `A`


Para las divisiones, podemos arregarlo a la Matriz o mantenerlo como un arreglo separado

guardar indices del pivote, tupla, dos numeros, etc.

Y con estas representaciones, que mencionaron, que tipo de operaciones podemos hacer para el metodo simplex?

`importar scipy linprog`

sumas, restas, escalamiento, la mayoria de operaciones matriciales

## Primeros pasos de la implementacion

Ahora que tenemos las ideas puestas, podemos pasar a implementar el metodo simplex en su lenguaje de programacion preferido ~~python~~. Para esto tenemos que pensar en que orden vamos a implementar los metodos. 

In [ ]:
import numpy as np
from typing import Any, List
from typing_extensions import Self
from dataclasses import dataclass

from typing import Tuple


@dataclass
class Solucion:
    indice: Tuple[int, int]
    valor_RHS: float
    nombre: str  # var_1, x_2


class Simplex:
    """
    El metodo simplex creado por estudiantes de la clase de PL de la ENES Morelia 2022 c:
    """

    def __call__(self, A: np.ndarray, w=False) -> List[Solucion]:

        self.A = A
        self.A.astype(float)

        n, m = self.A.shape
        self.pivotes = []
        if w:
            self.__calc_w()
            self.__simplex()
            # elimiar algo (W)

        self.__simplex()
        print(self)

        return self.__construir_sols()

    def __construir_sols(self) -> List[Solucion]:
        sols = []
        for i, j in self.pivotes:
            sols.append(Solucion((i, j), self.A[i, -1], f"x_{j + 1}"))
        return sols

    def eval_z(self):
        variables = np.array([self.A[i, -1] for i, _ in self.pivotes])
        return np.dot(variables, self.A[-1, :-1])

    def __calc_w(self):
        #"""
        #Su funcion para calcular la funcion w aqui
        #"""
        #...
        #def __aniadir_w(self):

        self.A = np.append(A, [A[-1,:]-sum(A)], axis=0)

    def __revisar_columnas_negativas(self) -> bool:
        n, m = self.A.shape
        for col in range(m - 1):
            if self.A[0][col] >= 0:
                continue

            count = 0

            for row in range(n - 1):
                if self.A[row, col] <= 0:
                    count += 1

            if count == n - 1:
                return True
        return False

    def __condicion_de_paro(self) -> bool:
        return (
            all(elem >= 0 for elem in self.A[-1, :-1])
            or self.__revisar_columnas_negativas()
        )

    #def __es_pivote(self) -> List[Tuple[int, int]]:
     #   """su funcion de tarea""" 
      #  ...

    def __es_pivote_ayuda(self, columna):
        """
        funcion para saber si cierta columna es un pivote y en 
        su caso regresar su índice, si no lo es regresar False
        
        Regresa
        --------------
        el índice de fila en caso de que la columna sí sea pivote
        """
        uno_y_ceros = [0,0]
        fila_uno = None
        for i,el in enumerate(columna):
          if el == 0:
            uno_y_ceros[0] += 1
          elif el == 1:
            uno_y_ceros[1] += 1
            fila_uno = i
          else:
            return False
        if uno_y_ceros[0] == len(columna)-1 and uno_y_ceros[1] == 1:
          return fila_uno
        return False
        
    def __es_pivote(self):
        """
        funcion para saber quienes son los pivotes
        
        Regresa
        --------------
        los indices de quienes son pivotes
        """
        self.VB = []
        for m in range(self.A.shape[1]):
          pivote_n = self.__es_pivote_ayuda(self.A[:,m])
          if pivote_n:
            self.VB.append([pivote_n,m])
        return self.VB


    def __sacar_pivote(self) -> Tuple[int, int]:
        # sacar el indice mas pequeño de Z
        col_index = np.argmin(self.A[-1])
        # dividr los indices de la col por el RHS
        divisiones = []

        for elem in self.A[:-1]:
            divisiones.append(
                np.inf if elem[col_index] <= 0 else elem[-1] / elem[col_index]
            )

        fila_index = np.argmin(divisiones)

        if divisiones.count(divisiones[fila_index]) > 1:
            fila_index = np.argmin(self.A[:-1, -1])

        return fila_index, col_index

    def __operaciones_elementales(self, fila_index: int, col_index: int):
        dividendo = self.A[fila_index, col_index]

        self.A[fila_index] /= dividendo

        for i in range(len(self.A)):
            if i == fila_index:
                continue

            elem = self.A[i, col_index]
            lista_aux = [elem * _ for _ in self.A[fila_index]]
            self.A[i] = [
                (dividendo * self.A[i, _]) - lista_aux[_] for _ in range(len(lista_aux))
            ]

    def __simplex(self):
        self.pivotes = self.__es_pivote()
        while not self.__condicion_de_paro():
            pivote_i, pivote_j = self.__sacar_pivote()
            self.__operaciones_elementales(pivote_i, pivote_j)
            # self.__intercambio(pivote_i, pivote_j)
            self.pivotes = (
                self.__es_pivote()
            )  # estamos reemplazando los pivotes en lugar de entra variable y sale variable

    def __repr__(self) -> str:
        return " ".join(
            f"x_{self.pivotes[i][1] + 1} = {self.A[self.pivotes[i][0]][-1]}"
            for i in range(len(self.pivotes))
        )


In [ ]:
tmp = np.array(
    [[4, 2, 1, 0, 0, 0], [5, 3, 0, -1, 1, 0], [2, 0, 0, 0, 0, 1], [15, 6, 0, 0, 0, 0]]
)
tmp

array([[ 4,  2,  1,  0,  0,  0],
       [ 5,  3,  0, -1,  1,  0],
       [ 2,  0,  0,  0,  0,  1],
       [15,  6,  0,  0,  0,  0]])

In [ ]:
simplex = Simplex()
sols =simplex(tmp)

x_5 = 0 x_6 = 1


In [ ]:
sols

[Solucion(indice=(1, 4), valor_RHS=0, nombre='x_5'),
 Solucion(indice=(2, 5), valor_RHS=1, nombre='x_6')]

In [ ]:
matriz_examen = np.array([
    [4,2,1,0,0,0, 24],
    [5,3,0,-1,1,0, 30],
    [2,0,0,0,0,1, 8],
    [15,6,0,0,0,0,0]
])
simplex = Simplex()
sols = simplex(matriz_examen)


x_5 = 30 x_6 = 8


In [ ]:
sols

[Solucion(indice=(1, 4), valor_RHS=30, nombre='x_5'),
 Solucion(indice=(2, 5), valor_RHS=8, nombre='x_6')]

In [ ]:
m1 = np.array([
    [2,1,1,0,120],
    [1,1,0,1,90],
    [-80, -50, 0, 0, 0]
], dtype = float)

simplex = Simplex()
sols = simplex(m1)

# x_1 = 30 x_2 = 60

x_2 = 60.0


In [ ]:
sols

[Solucion(indice=(1, 1), valor_RHS=60.0, nombre='x_2')]

In [ ]:
m2 = np.array([
    [3,3,1,0,120],
    [6, 3, 0, 1, 180],
    [-400, -300, 0, 0, 0]
], dtype=float)

simplex = Simplex()
simplex(m2)

## x_1 = 20 x_2 = 20

[]

In [ ]:
m3 = np.array([
    [-2, 0, 6,  2, 0, -3, 1, 20],
    [-4, 1, 7,  1, 0, -1, 0, 10],
    [0, 0, -5,  3, 1, -1, 0, 60],
    [0, 0, 13, -6, 0,  2, 0, 110]
], dtype=float)

simplex = Simplex()
simplex(m3)

x_2 = 10.0 x_5 = 60.0


[Solucion(indice=(1, 1), valor_RHS=10.0, nombre='x_2'),
 Solucion(indice=(2, 4), valor_RHS=60.0, nombre='x_5')]

## Cosas para probar el simplex ya con $w$
Ejemplo de la pagina 95

In [ ]:
#aqui hay que calcular la w
matrix = np.array([
    [1,-2,-3,-2,3],
    [1,-1,2,1,11],
    [2,-3,1,1,0]
])

In [ ]:
A = np.random.random(size=(3, 3))
A

array([[0.57054904, 0.78699798, 0.35989446],
       [0.00513952, 0.38459778, 0.30962712],
       [0.98535879, 0.66555649, 0.24309702]])

In [ ]:
A.shape

(3, 3)

In [ ]:
for ix, fila in enumerate(A):
    fila += 1

A

array([[1.63524382, 1.07494141, 1.34365397],
       [1.22596319, 1.49433638, 1.34357011],
       [1.38035718, 1.33311104, 1.56075353]])

In [ ]:
A = [1, 2, 3, 4, 4, 5, -1]
np.argmin(A)
# A.index(min(A))

6

In [ ]:
A = np.array(A)
np.argmin(A)

6

In [ ]:
lista = [(1,1), (2,1),  (3,1)]

In [ ]:
for i,j in lista:
    print(f"{i = }, {j =}")

i = 1, j =1
i = 2, j =1
i = 3, j =1
